In [1]:
import torch
import numpy as np

In [3]:
from utils.make_data import autoregressive, generate_autoregressive_forecast_dataset
from models.conformal import nonconformity, cover, ConformalForecaster

In [4]:
train_dataset = generate_autoregressive_forecast_dataset(n_samples=1000, seq_len=100, horizon=10)
calibration_dataset = generate_autoregressive_forecast_dataset(n_samples=1000, seq_len=100, horizon=10)
test_dataset = generate_autoregressive_forecast_dataset(n_samples=100, seq_len=100, horizon=10)

In [26]:
def generate_autoregressive_forecast_dataset(n_samples=100,
                                             seq_len=100,
                                             n_features=1,
                                             X_mean=1,
                                             X_variance=2,
                                             noise_profile=None,
                                             memory_factor=0.9,
                                             mode="time-dependent",
                                             horizon=10):
    total_seq_len = seq_len + horizon
    # Create the input features of the generating process
    X_gen = [np.random.normal(X_mean, X_variance, (total_seq_len,
                                                   n_features))
             for _ in range(n_samples)]
    w = np.array([memory_factor ** k for k in range(total_seq_len)])

    if noise_profile is None:
        # default increasing noise profile
        noise_profile = np.array(
            [1 / (seq_len - 1) * k for k in range(total_seq_len)])

    X = None  # X stores the time series values generated from features X_gen
    if mode == "noise-sweep":
        X = torch.FloatTensor(
            [[(autoregressive(X_gen[k], w).reshape(total_seq_len, n_features) +
               np.random.normal(0, noise_profile[u], (total_seq_len,
                                                      n_features)))
                  .reshape(total_seq_len, ) for k in range(n_samples)]
             for u in range(len(noise_profile))])


    elif mode == "time-dependent":
        X = torch.FloatTensor([(autoregressive(X_gen[k], w)
              .reshape(total_seq_len, n_features) + (
                  torch.normal(mean=0.0, std=torch.tensor(noise_profile)))
              .detach().numpy().reshape(-1, n_features)).reshape(
                total_seq_len, )
                for k in range(n_samples)])
        

    # TODO clean up (un)squeezing.
    Y = torch.FloatTensor(X[:, -horizon:])  # `horizon` of predictions
    X = torch.nn.utils.rnn.pad_sequence(X[:, :-horizon], batch_first=True).unsqueeze(dim=-1)

    dataset = torch.utils.data.TensorDataset(X, Y)
    return dataset

In [173]:
import numpy as np
import torch
from torch import nn


def nonconformity(output, target):
    """Measures the nonconformity between output and target time series."""
    # Average MAE loss for every step in the sequence.
    return torch.nn.functional.l1_loss(output, target, reduction='none')


class ConformalForecaster(nn.Module):
    def __init__(self, embedding_size, input_size=1, horizon=1, alpha=0.05):
        super(ConformalForecaster, self).__init__()
        # input_size indicates the number of features in the time series
        # input_size=1 for univariate series.

        # Encoder and forecaster can be the same (if embeddings are
        # trained on `horizon`-step forecasts), but different models are
        # possible.

        # TODO try separate encoder and forecaster models.
        # TODO try the RNN autoencoder trained on reconstruction error.
        self.encoder = None

        # Single-shot multi-output univariate time series forecaster.
        # https://www.tensorflow.org/tutorials/structured_data/time_series#rnn_2
        # TODO consider autoregressive multi-output model:
        # https://www.tensorflow.org/tutorials/structured_data/time_series#advanced_autoregressive_model
        self.forecaster_rnn = nn.LSTM(input_size=input_size,
                                      hidden_size=embedding_size,
                                      batch_first=True)
        self.forecaster_out = nn.Linear(embedding_size, horizon)

        self.num_train = None
        self.calibration_scores = None
        self.critical_calibration_score = None
        self.alpha = alpha

    def forward(self, x):
        _, (h_n, c_n) = self.forecaster_rnn(x)
        out = self.forecaster_out(h_n)

        return out.squeeze(dim=0)

    def fit(self, dataset, calibration_dataset, epochs, lr, batch_size=150):
        # Train the forecaster to return correct multi-step predictions.
        train_loader = torch.utils.data.DataLoader(dataset,
                                                   batch_size=batch_size,
                                                   shuffle=True)
        self.num_train = len(dataset)

        optimizer = torch.optim.Adam(self.parameters(), lr=lr)
        criterion = torch.nn.MSELoss()

        for epoch in range(epochs):
            self.train()
            train_loss = 0.

            for sequences, targets in train_loader:  # iterate through batches
                optimizer.zero_grad()

                out = self(sequences)

                loss = criterion(out, targets)
                loss.backward()

                train_loss += loss.item()

                optimizer.step()

            mean_train_loss = train_loss / len(train_loader)
            if epoch % 50 == 0:
                print('Epoch: {}\tTrain loss: {}'.format(epoch, mean_train_loss))

        # Collect calibration scores
        self.calibrate(calibration_dataset)

    def calibrate(self, calibration_dataset):
        """
        Computes the nonconformity scores for the calibration dataset.
        """
        calibration_loader = torch.utils.data.DataLoader(calibration_dataset,
                                                         batch_size=1)
        calibration_scores = []

        with torch.set_grad_enabled(False):
            self.eval()
            for sequences, targets in calibration_loader:
                out = self(sequences)
                calibration_scores.extend(nonconformity(out, targets).detach().numpy())

        self.calibration_scores = torch.tensor(calibration_scores).T

        # Given p_{z}:=\frac{\left|\left\{i=m+1, \ldots, n+1: R_{i} \geq R_{n+1}\right\}\right|}{n-m+1}
        # and the accepted R_{n+1} = \Delta(y, f(x_{test})) are such that
        # p_{z} > \alpha we have that the nonconformity scores should be below
        # the (corrected) alpha% of calibration scores.
        self.critical_calibration_scores = torch.tensor([np.quantile(
            position_calibration_scores, q=1 - self.alpha * self.num_train / (self.num_train + 1))
            for position_calibration_scores in self.calibration_scores])

    def predict(self, x):
        """Forecasts the time series with conformal uncertainty intervals."""
        out = self(x)
        # TODO +/- nonconformity will not return *adaptive* interval widths.
        return torch.vstack([out - self.critical_calibration_scores,
                             out + self.critical_calibration_scores]).T


In [5]:
model = ConformalForecaster(embedding_size=8, horizon=10, error_rate=0.05)

In [6]:
model.fit(train_dataset, calibration_dataset, epochs=100, lr=0.01, batch_size=100)

Epoch: 0	Train loss: 117.00947952270508
Epoch: 50	Train loss: 15.443363380432128


In [7]:
test_dataset = generate_autoregressive_forecast_dataset(n_samples=100, seq_len=100, horizon=10)
model.eval()
c = []
for sequences, target in test_dataset:
    sequences = sequences.unsqueeze(dim=0)
    out = model(sequences).squeeze()
    pred = torch.vstack([out - model.critical_calibration_scores,
                         out + model.critical_calibration_scores]).T
    c.append(cover(pred, target))
print('Achieved coverage: {}'.format(np.mean(c)))

Achieved coverage: 0.72


In [164]:
alpha = 0.05
num_train = 1000

def calibrate(calibration_dataset):
    """
    Computes the nonconformity scores for the calibration dataset.
    """
    calibration_loader = torch.utils.data.DataLoader(calibration_dataset,
                                                     batch_size=1)
    calibration_scores = []

    with torch.set_grad_enabled(False):
        model.eval()
        for sequences, targets in calibration_loader:
            out = model(sequences)
            calibration_scores.extend(nonconformity(out, targets).detach().numpy())

    calibration_scores = torch.tensor(calibration_scores).T

    # Given p_{z}:=\frac{\left|\left\{i=m+1, \ldots, n+1: R_{i} \geq R_{n+1}\right\}\right|}{n-m+1}
    # and the accepted R_{n+1} = \Delta(y, f(x_{test})) are such that
    # p_{z} > \alpha we have that the nonconformity scores should be below
    # the (corrected) alpha% of calibration scores.
    critical_calibration_scores = torch.tensor([np.quantile(
        position_calibration_scores, q= alpha * num_train / (num_train + 1))
        for position_calibration_scores in calibration_scores])
    print(critical_calibration_scores)

In [165]:
calibrate(calibration_dataset)

tensor([0.1335, 0.2231, 0.2458, 0.2786, 0.2465, 0.2878, 0.3393, 0.3049, 0.2977,
        0.3465], dtype=torch.float64)


In [176]:
1 - alpha * num_train / (num_train + 1)

0.9500499500499501